In [1]:
## This contains a pure IK based troting motion plan
## Author : Avadesh Meduri
## Date : 26/02/2021

import time
import numpy as np
import pinocchio as pin
import crocoddyl

from robot_properties_solo.config import Solo12Config
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator

In [18]:
dt = 1e-2
T = 2.0

robot = Solo12Config.buildRobotWrapper()
q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot.model.nv)])

pin.forwardKinematics(robot.model, robot.data, q0, pin.utils.zero(robot.model.nv))
pin.updateFramePlacements(robot.model, robot.data)

fl_loc = robot.data.oMf[robot.model.getFrameId("FL_FOOT")].translation
fr_loc = robot.data.oMf[robot.model.getFrameId("FR_FOOT")].translation
hl_loc = robot.data.oMf[robot.model.getFrameId("HL_FOOT")].translation
hr_loc = robot.data.oMf[robot.model.getFrameId("HR_FOOT")].translation


In [3]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [19]:
sl = np.array([0.0, 0.0, 0])
mass = pin.computeTotalMass(robot.model, robot.data)
com_loc = pin.centerOfMass(robot.model, robot.data, q0)
ctraj = np.zeros((int(T/dt), 6))
ctraj[:,0] = 3*mass*sl[0]/T

In [20]:
gg = GaitGenerator(robot, T, dt)

# gg.create_centroidal_task(ctraj, 0, 2.0, "cmom_task", 1e4)

gg.create_swing_foot_task(fl_loc, fl_loc + sl, 0.0, 0.5, 0.1, "FL_FOOT", "FL_ftc1", 1e2)
gg.create_swing_foot_task(hr_loc, hr_loc + sl, 0.0, 0.5, 0.1, "HR_FOOT", "HR_ftc1", 1e2)
gg.create_contact_task(fr_loc, 0.0, 0.5, "FR_FOOT", "FR_ctc1", 1e3)
gg.create_contact_task(hl_loc, 0.0, 0.5, "HL_FOOT", "HL_ctc1", 1e3)

gg.create_swing_foot_task(fr_loc, fr_loc + 2*sl, 0.5, 1.0, 0.1, "FR_FOOT", "FR_ftc1", 1e2)
gg.create_swing_foot_task(hl_loc, hl_loc + 2*sl, 0.5, 1.0, 0.1, "HL_FOOT", "HL_ftc1", 1e2)
gg.create_contact_task(fl_loc + sl, 0.5, 1.0, "FL_FOOT", "FL_ctc1", 1e3)
gg.create_contact_task(hr_loc + sl, 0.5, 1.0, "HR_FOOT", "HR_ctc1", 1e3)

gg.create_swing_foot_task(fl_loc + sl, fl_loc + 3*sl, 1.0, 1.5, 0.1, "FL_FOOT", "FL_ftc1", 1e2)
gg.create_swing_foot_task(hr_loc + sl, hr_loc + 3*sl, 1.0, 1.5, 0.1, "HR_FOOT", "HR_ftc1", 1e2)
gg.create_contact_task(fr_loc + 2*sl, 1.0, 1.5, "FR_FOOT", "FR_ctc2", 1e3)
gg.create_contact_task(hl_loc + 2*sl, 1.0, 1.5, "HL_FOOT", "HL_ctc2", 1e3)

gg.create_swing_foot_task(fr_loc + 2*sl, fr_loc + 4*sl, 1.5, 2.0, 0.1, "FR_FOOT", "FR_ftc1", 1e2)
gg.create_swing_foot_task(hl_loc + 2*sl, hl_loc + 4*sl, 1.5, 2.0, 0.1, "HL_FOOT", "HL_ftc1", 1e2)
gg.create_contact_task(fl_loc + 3*sl, 1.5, 2.0, "FL_FOOT", "FL_vtc2", 1e3)
gg.create_contact_task(hr_loc + 3*sl, 1.5, 2.0, "HR_FOOT", "HR_vtc2", 1e3)

# comTrack = crocoddyl.CostModelCoMPosition(gg.ik.state, 4*sl + com_loc, gg.ik.state.nv)
# gg.ik.terminalCostModel.addCost("comtask", comTrack, 1e+3)

xs = gg.optimize(x0)


In [21]:
for i in range(len(xs)):
    time.sleep(0.01)
    viz.display(xs[i][:robot.model.nq])